# Troubleshooting Authentication with DefaultAzureCredential

## Overview

DefaultAzureCredential is the recommended way to handle authentication in Azure applications. It provides a streamlined authentication flow by trying multiple credential types in sequence until one succeeds. This notebook will help you troubleshoot common authentication issues and ensure proper setup.

## Understanding DefaultAzureCredential

DefaultAzureCredential attempts authentication methods in the following order:

1. Environment Credentials
2. Workload Identity (in Kubernetes)
3. Managed Identity
4. Azure CLI Credentials
5. Azure PowerShell Credentials
6. Visual Studio Code Credentials
7. Interactive Browser Authentication (as fallback)

## Prerequisites

Ensure you have the following installed:
- Azure CLI
- Azure Developer CLI (optional)
- Python Virtual Environment or Conda (use `uv venv` or `conda create`)
- Required role assignments (Azure AI Developer)
- Jupyter Notebook environment - kernel configured to use Python 3.8 or later

## Authentication Methods

### 1. Using Azure CLI (Recommended for Local Development)

In [ ]:
# Install required packages
!pip install azure-identity

# First, we'll authenticate using Azure CLI
This is the recommended approach for local development.

When you run the code below, you will be redirected to:
- Either the Azure portal in your browser to complete the login 
- Or use Windows login if you're already signed in to your machine

The code will:
1. Load environment variables from .env file, including the TENANT_ID
2. Use Azure CLI to log in to your specific tenant  
3. Test authentication by attempting to get a token


In [ ]:
# Import required packages
from IPython.display import display
from IPython.display import HTML
import getpass
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Get tenant ID from environment variable
tenant_id = os.getenv("TENANT_ID")

# Azure login with specific tenant
!az login --tenant {tenant_id}

# Next, we'll test the authentication by attempting to get a token using DefaultAzureCredential

DefaultAzureCredential will try multiple authentication methods in this order:
1. Environment credentials (if environment variables are set)
2. Managed Identity credentials (if running in Azure)
3. Shared Token Cache credentials (from previous logins) 
4. Visual Studio Code credentials (if using VS Code)
5. Azure CLI credentials (which we just set up)

The code below will:
1. Create a DefaultAzureCredential instance
2. Try to get a token for Azure Cognitive Services
3. Print success message if token is acquired

>Note: You may see some warning/error messages as it tries different authentication methods - 
>this is normal and can be ignored as long as you see "Successfully acquired token!" at the end


In [ ]:
# Then use DefaultAzureCredential in your code
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
import logging

# Enable detailed logging
logging.basicConfig(level=logging.DEBUG)

try:
    credential = DefaultAzureCredential()
    # Test token acquisition
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully acquired token!")
except Exception as e:
    print(f"Authentication failed: {str(e)}")

# Now that you have successfully authenticated, you can proceed to [1-environment_setup.ipynb](1-environment_setup.ipynb), or try the additional authentication methods or troubleshoot below.


### 2. Using Visual Studio Code (optional)

If you're using VS Code with the Azure extension:

In [ ]:
from azure.identity import DefaultAzureCredential, VisualStudioCodeCredential

try:
    # Explicitly try VS Code credentials
    vscode_credential = VisualStudioCodeCredential()
    token = vscode_credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully authenticated with VS Code credentials!")
except Exception as e:
    print(f"VS Code authentication failed: {str(e)}")

### 3. Using Service Principal (Optional - have to set environment variables)

In [ ]:
import os

# Set these environment variables before running your application
required_env_vars = {
    "AZURE_CLIENT_ID": "your-client-id",
    "AZURE_CLIENT_SECRET": "your-client-secret",
    "AZURE_TENANT_ID": "your-tenant-id"
}

# Verify environment variables are set
def check_env_vars():
    missing_vars = [var for var, _ in required_env_vars.items() 
                   if not os.getenv(var)]
    if missing_vars:
        print(f"Missing environment variables: {', '.join(missing_vars)}")
        return False
    return True

if check_env_vars():
    credential = DefaultAzureCredential()
    # Test authentication
    try:
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print("Successfully authenticated using environment credentials!")
    except Exception as e:
        print(f"Authentication failed: {str(e)}")

## Troubleshooting Steps

### 1. Verify Role Assignments

```bash
# Check role assignments for your user/service principal
az role assignment list --assignee "your-email@domain.com" --output table
```

### 2. Debug Token Acquisition


In [ ]:
import logging
from azure.identity import DefaultAzureCredential

# Set up detailed logging
logger = logging.getLogger('azure.identity')
logger.setLevel(logging.DEBUG)

# Use a basic StreamHandler instead of LoggingHandler
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

def test_authentication():
    try:
        credential = DefaultAzureCredential(logging_enable=True)
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print(f"Authentication successful!")
        return True
    except Exception as e:
        print(f"Authentication failed with error: {str(e)}")
        print("\nTroubleshooting steps:")
        print("1. Verify you're logged in: 'az account show'")
        print("2. Check role assignments: 'az role assignment list'")
        print("3. Verify tenant ID: 'az account show --query tenantId'")
        return False

# Run the test
test_authentication()


### 3. Verify Azure AI Developer Role

In [ ]:
def verify_ai_developer_role(subscription_id, resource_group, resource_name):
    from azure.mgmt.authorization import AuthorizationManagementClient
    
    auth_client = AuthorizationManagementClient(
        credential=DefaultAzureCredential(),
        subscription_id=subscription_id
    )
    
    resource_id = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}"
    
    assignments = auth_client.role_assignments.list_for_scope(resource_id)
    
    ai_developer_role_id = "a97b65f3-24c7-4388-baec-2e87135dc908"  # Azure AI Developer role ID
    
    for assignment in assignments:
        if assignment.role_definition_id.endswith(ai_developer_role_id):
            return True
    
    return False

## Common Issues and Solutions

1. **Token Acquisition Failed**
   - Verify Azure CLI login: `az login --tenant <tenant-id>`
   - Check default subscription: `az account show`
   - Ensure correct tenant: `az account set --subscription <subscription-id>`

2. **Missing Role Assignments**
   - Add Azure AI Developer role:
   ```bash
   az role assignment create --assignee "user@domain.com" \
       --role "Azure AI Developer" \
       --scope "/subscriptions/<subscription-id>/resourceGroups/<resource-group>/providers/Microsoft.CognitiveServices/accounts/<resource-name>"
   ```

3. **Environment Variable Issues**
   - Verify environment variables are set correctly
   - Check for typos in variable names
   - Ensure no extra spaces in values

## Best Practices

1. Always use environment variables for service principal credentials
2. Implement proper error handling and logging
3. Use managed identities when deploying to Azure services
4. Regularly rotate service principal secrets
5. Follow the principle of least privilege when assigning roles

## Additional Resources

- [Azure Identity Documentation](https://docs.microsoft.com/python/api/overview/azure/identity-readme)
- [DefaultAzureCredential Authentication Flow](https://docs.microsoft.com/azure/developer/python/azure-sdk-authenticate)
- [Azure RBAC Documentation](https://docs.microsoft.com/azure/role-based-access-control/overview)